#### j11a210_T 커널로 작업

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

## 환경 설정

In [ ]:
!pip install AudioSegment
!pip install torch torchaudio
!pip install deepfilternet

In [ ]:
pip install --upgrade torchaudio

## import

In [ ]:
import torch
import torchaudio
from df import enhance, init_df
from IPython.display import Audio

## DeNoise 코드

In [ ]:
import time

def split_audio(audio_path, segment_length):
    # Load audio file
    waveform, sample_rate = torchaudio.load(audio_path)
    num_samples = waveform.shape[1]
    
    # Calculate total length in seconds
    total_length = num_samples / sample_rate
    
    # Split the audio into segments
    segments = []
    for start in range(0, int(total_length), segment_length):
        end = min(start + segment_length, int(total_length))
        start_sample = start * sample_rate
        end_sample = end * sample_rate
        segments.append(waveform[:, start_sample:end_sample])
    
    return segments, sample_rate

def enhance_audio(segments, model, df_state):
    enhanced_segments = []
    for segment in segments:
        # Convert segment to mono if needed
        segment = segment.mean(dim=0).unsqueeze(0)  # Convert to mono
        # Apply noise reduction
        enhanced_segment = enhance(model, df_state, segment)
        enhanced_segments.append(enhanced_segment)
    return enhanced_segments

def combine_audio(segments):
    return torch.cat(segments, dim=1)

start_time = time.time()

# Initialize the DeepFilterNet model
model, df_state, _ = init_df()

original = "../testAudio/noNoise_codeapple.wav"
# no_noise = "noNoise.wav"

# 노이즈가 포함된 오디오 파일 로드
noisy_audio_path = original
waveform, sample_rate = torchaudio.load(noisy_audio_path)
segment_length = 10 * 60  # 10 minutes in seconds


# 오디오를 단일 채널로 변환하고 필요한 경우 차원을 추가하여 형식 조정
waveform = waveform.mean(dim=0).unsqueeze(0)  # 모노로 변환

# 오디오 향상
# Step 1: Split audio
audio_segments, sample_rate = split_audio(original, segment_length)

# Step 2: Enhance audio
enhanced_segments = enhance_audio(audio_segments, model, df_state)

# Step 3: Combine audio
combined_audio = combine_audio(enhanced_segments)

# 노이즈 제거된 음원 저장
# torchaudio.save(no_noise, enhanced_waveform, sample_rate)

end_time = time.time() - start_time

print(f"Time: {end_time:.2f} seconds")


In [ ]:
# torchaudio.save("no_noise.wav", combined_audio, sample_rate) # 저장 코드
torch.cuda.empty_cache() # 캐시 메모리 삭제하고 사용

In [ ]:
def calculate_averages(segments):
    # 초기값 설정
    total_temperature = 0
    total_avg_logprob = 0
    total_compression_ratio = 0
    total_no_speech_prob = 0
    segment_count = len(segments)
    
    # 각 segment에서 값 추출 및 합산
    for segment in segments:
        total_temperature += segment.get('temperature', 0)
        total_avg_logprob += segment.get('avg_logprob', 0)
        total_compression_ratio += segment.get('compression_ratio', 0)
        total_no_speech_prob += segment.get('no_speech_prob', 0)
    
    # 평균 계산
    avg_temperature = total_temperature / segment_count if segment_count > 0 else 0
    avg_avg_logprob = total_avg_logprob / segment_count if segment_count > 0 else 0
    avg_compression_ratio = total_compression_ratio / segment_count if segment_count > 0 else 0
    avg_no_speech_prob = total_no_speech_prob / segment_count if segment_count > 0 else 0
    
    # 결과 출력
    print(f"Average Temperature: {avg_temperature}")
    print(f"Average Avg Log Probability: {avg_avg_logprob}")
    print(f"Average Compression Ratio: {avg_compression_ratio}")
    print(f"Average No Speech Probability: {avg_no_speech_prob}")
    
    return {
        'avg_temperature': avg_temperature,
        'avg_avg_logprob': avg_avg_logprob,
        'avg_compression_ratio': avg_compression_ratio,
        'avg_no_speech_prob': avg_no_speech_prob
    }



In [ ]:

# 평균 계산
averages = calculate_averages(no_Noise_result['segments'])
print("")
averages = calculate_averages(result['segments'])
